In [55]:
import pandas as pd
import dotenv
import os
from pinecone import Pinecone
from pinecone import ServerlessSpec
from pinecone_text.sparse import BM25Encoder
import time
import spacy
import json


pinecone_api = os.getenv('PINECONE_API_KEY')
rds_password = os.getenv('RDS_PASSWORD')
rds_host = os.getenv('RDS_HOST')
pc = Pinecone(api_key=pinecone_api)


In [5]:
import json
with open('data.json', 'r') as file:
    data = json.load(file)

dense_size = len(data[0]['vector'])
dense_size

1024

In [51]:
df = pd.read_csv('mnt/efs/data_pre_v3.csv')
df.head().to_csv('data_sample.csv')

In [9]:
from tqdm import tqdm

# Load the spaCy model once
nlp = spacy.load("pl_core_news_sm")

def preproc(text):
    text = text.replace('\n', ' ').replace('\t', ' ')
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(tokens)

# Assuming df is your DataFrame
tqdm.pandas()

# Apply the preprocessing function with a progress bar
df['Krótki opis preproc'] = df['Krótki opis'].progress_apply(preproc)

100%|██████████| 13240/13240 [17:03<00:00, 12.94it/s]


In [10]:
df.to_csv('mnt/efs/data_pre_v3.csv')

In [11]:
bm25 = BM25Encoder()

bm25.fit(df['Krótki opis preproc'])

sparse = bm25.encode_documents([text for text in df['Krótki opis preproc']])

  0%|          | 0/13240 [00:00<?, ?it/s]

100%|██████████| 13240/13240 [00:24<00:00, 551.43it/s]


In [20]:
sparse = [{'indices': item['indices'][:1000], 'values': item['values'][:1000]} for item in sparse]

In [12]:
dense = [item['vector'] for item in data]

In [21]:
meta_dict = [{'id':id} for id in df['uuid'].tolist()]

upserts = []
for _id, sparse_, dense_, meta in zip(df['uuid'].tolist(), sparse, dense, meta_dict):
    upserts.append({
            'id': _id,
            'sparse_values': sparse_,
            'values': dense_,
            'metadata': meta
        })

In [22]:
index_name = "tender-ai-hybrid-mmlw"
cloud = 'aws'
region = 'us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)
# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=dense_size,
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

In [23]:
# Function to split data into batches
def split_into_batches(data, batch_size):
    batches = [data[i:i + batch_size] for i in range(0, len(data), batch_size)]
    return batches

# Define your batch size
batch_size = 100

# Split upserts into batches
batches = split_into_batches(upserts, batch_size)

In [24]:
import tqdm

index = pc.Index(index_name)

for batch in tqdm.tqdm(batches):
    index.upsert(batch)

100%|██████████| 133/133 [11:27<00:00,  5.17s/it]


In [26]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('mnt/efs/model')

In [31]:
query = "MarkX Piotr Rajshttps://www.markx-rajs.pl/ budownictow drogowe; Drogi, nasypy stabilizacje kanalizacje, ale tez duzo innych- patrz strona. Lokalizacja: cala polska"
query = 'zapytanie: ' + query
filter_ids = df['uuid'].tolist()
# create sparse and dense vectors
sparse_query = bm25.encode_queries(query)
dense_query = model.encode(query, convert_to_tensor=False, show_progress_bar=False).tolist()
# search
result = index.query(
    top_k=5,
    filter={
        "id": {"$in":filter_ids},
    },
    vector=dense_query,
    sparse_vector=sparse_query,
    include_metadata=True
)

results = [{
    'id': result['id'],
    'opis': df[df['uuid'] == result['id']]["Krótki opis"].squeeze(),
} for result in result['matches']]
pd.DataFrame(data=results)

,id,opis
0,959118ca-e475-4f5e-b5fa-c27dd50500fb,Renowacja kluczowych elementów kościoła parafi...
1,698c4ef7-0c55-4276-8a67-5ebe87883803,Modernizacja istniejącego terenu magazynowego ...
2,a9c5bbf9-36e2-4a29-ba99-562dd65af607,"Remont budynku ""P"" we Wrocławiu przy ulicy Gaz..."
3,fadb19f0-abb7-4923-9daf-e70919d5b209,Remont budynku kotłowni i SUW w Tłoczni Gazu R...
4,193b5c97-cfd9-452c-a5fe-7126706d917c,Renowacja wieży kościelnej oraz zagospodarowan...


In [18]:
#pc.delete_index("tender-ai-hybrid-mmlw")

/var/folders/x1/2j32gjvd4v16s6kf306sfdcw0000gn/T/ipykernel_7409/3974689322.py:5: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [49]:
df = pd.read_csv('mnt/efs/data_pre_v3.csv')

In [60]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Date, ARRAY
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import numpy as np
# Define the SQLAlchemy base and model
Base = declarative_base()

class Tender(Base):
    __tablename__ = 'tenders'
    id = Column(Integer, primary_key=True, autoincrement=True)
    data_publikacji = Column(Date)
    data_zakonczenia = Column(Date)
    miasto_powiat = Column(String)
    zamawiajacy = Column(String)
    przedmiot_zamowienia = Column(String)
    link = Column(String)
    wojewodztwo = Column(ARRAY(String))
    krotki_opis = Column(String)
    krotki_opis_preproc = Column(String)
    glowny_kod_cpv = Column(String)
    okres_realizacji = Column(String)
    wymagane_wadium = Column(String)
    kategoria_colab = Column(String)
    uuid = Column(String)
    categories = Column(ARRAY(String))

# Replace these variables with your own information
rds_host = rds_host
rds_port = '5432'  # usually 5432
rds_dbname = 'postgres'
rds_username = 'postgres'
rds_password = rds_password

# Create a connection string
DATABASE_URL = f"postgresql+psycopg2://{rds_username}:{rds_password}@{rds_host}:{rds_port}/{rds_dbname}"

# Create the database engine
engine = create_engine(DATABASE_URL)
Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Load the CSV file
file_path = 'mnt/efs/data_pre_v3.csv'  # Update this with your file path
df = pd.read_csv(file_path)

# Preprocess the data
df['Data Publikacji'] = pd.to_datetime(df['Data Publikacji']).dt.date
df['Data Zakończenia'] = pd.to_datetime(df['Data Zakończenia']).dt.date
df['Województwo'] = df['Województwo'].apply(lambda x: x.split(';') if pd.notnull(x) else [])
df['categories'] = df['categories'].apply(lambda x: x.split(';') if pd.notnull(x) else [])
df['Wymagane Wadium'] = df['Wymagane Wadium'].replace({np.nan: None}).astype(object)

# Clean string columns
string_columns = [
    'Miasto/Powiat', 'Zamawiający', 'Przedmiot zamówienia', 'Link',
    'Krótki opis', 'Krótki opis preproc', 'Główny kod CPV',
    'Okres realizacji', 'kategoria - colab', 'uuid'
]
for col in string_columns:
    df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)

# Ensure 'kategoria - colab' column is treated as a string
df['kategoria - colab'] = df['kategoria - colab'].astype(str)

# Insert data into the database
for _, row in df.iterrows():
    tender = Tender(
        data_publikacji=row['Data Publikacji'],
        data_zakonczenia=row['Data Zakończenia'],
        miasto_powiat=row['Miasto/Powiat'],
        zamawiajacy=row['Zamawiający'],
        przedmiot_zamowienia=row['Przedmiot zamówienia'],
        link=row['Link'],
        wojewodztwo=row['Województwo'],
        krotki_opis=row['Krótki opis'],
        krotki_opis_preproc=row['Krótki opis preproc'],
        glowny_kod_cpv=row['Główny kod CPV'],
        okres_realizacji=row['Okres realizacji'],
        wymagane_wadium=row['Wymagane Wadium'],
        kategoria_colab=row['kategoria - colab'],
        uuid=row['uuid'],
        categories=row['categories']
    )
    session.add(tender)

session.commit()

# Close the session
session.close()

/var/folders/x1/2j32gjvd4v16s6kf306sfdcw0000gn/T/ipykernel_7409/4118990035.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [78]:
# Create the database engine
engine = create_engine(DATABASE_URL)

def format_query(locations, categories):
    # Convert lists to PostgreSQL array strings
    locations_str = ', '.join(f"'{loc}'" for loc in locations)
    categories_str = ', '.join(f"'{cat}'" for cat in categories)
    
    # Construct the query string
    query = f'''SELECT *
FROM tenders
WHERE wojewodztwo::text[] && ARRAY[{locations_str}]::text[]
AND categories::text[] && ARRAY[{categories_str}]::text[]'''
    
    return query

# Example usage
locations = ['mazowieckie']
categories = ['budownictwo - drogi chodniki place', 'budownictwo - wodno-kanalizacyjne', 'budownictwo - sanitarno-hydrauliczne']
query = format_query(locations, categories)


# Execute the query and load the data into a pandas DataFrame
df = pd.read_sql_query(query, engine)

# Display the DataFrame
ids = df['uuid'].tolist()

query = "MarkX Piotr Rajshttps://www.markx-rajs.pl/ budownictow drogowe; Drogi, nasypy stabilizacje kanalizacje, ale tez duzo innych- patrz strona. Lokalizacja: cala polska"
dense_query = 'zapytanie: ' + query
filter_ids = ids
# create sparse and dense vectors
sparse_query = preproc(query)
sparse_query = bm25.encode_queries(sparse_query)
dense_query = model.encode(dense_query, convert_to_tensor=False, show_progress_bar=False).tolist()
# search
result = index.query(
    top_k=20,
    filter={
        "id": {"$in":filter_ids},
    },
    vector=dense_query,
    sparse_vector=sparse_query,
    include_metadata=True
)

result


{'matches': [{'id': 'fadb19f0-abb7-4923-9daf-e70919d5b209',
              'metadata': {'id': 'fadb19f0-abb7-4923-9daf-e70919d5b209'},
              'score': 270.68515,
              'values': []},
             {'id': '3010ea92-1926-4033-aa60-25ed40931766',
              'metadata': {'id': '3010ea92-1926-4033-aa60-25ed40931766'},
              'score': 260.2034,
              'values': []},
             {'id': 'f3ca6137-7705-495b-a1bc-b5a1807906d0',
              'metadata': {'id': 'f3ca6137-7705-495b-a1bc-b5a1807906d0'},
              'score': 257.731354,
              'values': []},
             {'id': '31e64c4e-6300-47aa-a182-1d017a233406',
              'metadata': {'id': '31e64c4e-6300-47aa-a182-1d017a233406'},
              'score': 249.297363,
              'values': []},
             {'id': '05b06f7a-ae41-4877-bede-7d9326b15cbb',
              'metadata': {'id': '05b06f7a-ae41-4877-bede-7d9326b15cbb'},
              'score': 248.326889,
              'values': []},
             